In [ ]:
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_percentage_error, \
    mean_absolute_error
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import grad

# --- Set GPU Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Define Navier-Stokes PINN Model ---
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))

    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            x = torch.tanh(layer(x))
        x = self.layers[-1](x)  # Output layer (no activation)
        return x

# --- Physics-Informed Loss Function ---
def navier_stokes_loss(model, x, viscosity=0.01):
    x.requires_grad = True
    u_pred = model(x)[:, 0]
    v_pred = model(x)[:, 1]
    p_pred = model(x)[:, 2]

    du_dx = grad(u_pred, x, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0][:, 0]
    dv_dy = grad(v_pred, x, grad_outputs=torch.ones_like(v_pred), create_graph=True)[0][:, 1]
    dp_dx = grad(p_pred, x, grad_outputs=torch.ones_like(p_pred), create_graph=True)[0][:, 0]
    dp_dy = grad(p_pred, x, grad_outputs=torch.ones_like(p_pred), create_graph=True)[0][:, 1]

    # Continuity equation (divergence-free condition)
    continuity = du_dx + dv_dy

    # Momentum equations
    momentum_x = u_pred * du_dx + v_pred * dv_dy + dp_dx - viscosity * grad(du_dx, x, torch.ones_like(du_dx), create_graph=True)[0][:, 0]
    momentum_y = u_pred * dv_dy + v_pred * dv_dy + dp_dy - viscosity * grad(dv_dy, x, torch.ones_like(dv_dy), create_graph=True)[0][:, 1]

    # Compute loss
    loss = torch.mean(continuity**2) + torch.mean(momentum_x**2) + torch.mean(momentum_y**2)
    return loss

# --- Generate Training Data for Circular Airfoil ---
def generate_airfoil_data(n_points=5000):
    theta = np.linspace(0, 2 * np.pi, n_points)
    x = np.cos(theta)  # Circular airfoil boundary
    y = np.sin(theta)
    return torch.tensor(np.stack([x, y], axis=1), dtype=torch.float32).to(device)

# --- Train PINN ---
def train_pinn(model, optimizer, epochs=5000):
    losses = []
    x_train = generate_airfoil_data()

    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = navier_stokes_loss(model, x_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        if epoch % 500 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.6f}")

    return losses

# --- Define Model and Train ---
pinn_model = PINN([2, 50, 50, 50, 3]).to(device)
optimizer = torch.optim.Adam(pinn_model.parameters(), lr=0.001)
losses = train_pinn(pinn_model, optimizer)

# --- Plot Training Loss ---
plt.figure(figsize=(8, 6))
plt.plot(losses, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("PINN Training Loss Over Time")
plt.legend()
plt.grid(True)
plt.show()

# --- Predict Flow Field Over Airfoil ---
def predict_flow(model, n_points=1000):
    x_test = generate_airfoil_data(n_points)
    model.eval()
    with torch.no_grad():
        u_pred, v_pred, p_pred = model(x_test).cpu().numpy().T
    return x_test.cpu().numpy(), u_pred, v_pred, p_pred

# --- Visualization of Flow Field ---
x_test, u_pred, v_pred, p_pred = predict_flow(pinn_model)
plt.figure(figsize=(8, 6))
plt.quiver(x_test[:, 0], x_test[:, 1], u_pred, v_pred, color='blue')
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Predicted Velocity Field Over Airfoil")
plt.grid(True)
plt.show()

# --- Evaluate Model Performance ---
y_true = np.zeros_like(p_pred)  # Placeholder true values (for comparison)
r2 = r2_score(y_true, p_pred)
mse = mean_squared_error(y_true, p_pred)
print(f"Model Performance: R² = {r2:.4f}, MSE = {mse:.6f}")


mae_piml = mean_absolute_error(y_true, p_pred)
mape_piml = mean_absolute_percentage_error(y_true, p_pred)
mse_piml = mean_squared_error(y_true, p_pred)
r2_piml = r2_score(y_true, p_pred)
accuracy_piml = accuracy_score((p_pred > 0).astype(int), (p_pred > 0).astype(int))

print(f"PIML - MAE: {mae_piml:.6f}, MAPE: {mape_piml:.6f}, MSE: {mse_piml:.6f}, R²: {r2_piml:.6f}, Accuracy: {accuracy_piml:.6f}")